In [30]:
import requests

from credentials.credentials import SMALLCASE_COOKIE, SMALLCASE_JWT, CSRF_TOKEN, APP_INSTALLATION_ID

ImportError: cannot import name 'CSRF_TOKEN' from 'credentials.credentials' (/Users/aaryanpagar/Desktop/CODING/laxmi-chit-fund/notebooks/credentials/credentials.py)

In [29]:
headers = {
    "Origin": "https://smallcase.zerodha.com", 
    "Referer": "https://smallcase.zerodha.com/",
    "Accept": "application/json, text/plain, */*",
    "X-Sc-Broker": "kite",
    "X-Sc-Client-Type": "web",
    "X-Sc-App-Version": "1.0.0",
    "Cookie": SMALLCASE_COOKIE,
    "X-Sc-Jwt": SMALLCASE_JWT,
    "X-Csrf-Token": CSRF_TOKEN,
    "X-Sc-App-Install-Id": APP_INSTALLATION_ID,
}

NameError: name 'CSRF_TOKEN' is not defined

In [17]:
resp = requests.get("https://api.smallcase.com/smallcases/smallcase?scid=SCNM_0025", headers=headers)

print(resp.json())

{'success': True, 'errors': [], 'data': {'_id': '5a0b0604f86b54456739e5c0', 'version': 26, 'rationale': '<p>Electric vehicles (EVs) are gaining popularity due to increasing concerns over environmental pollution, sustainability and rising fuel prices. Technological advancements and Govt initiatives to promote clean energy and sustainable transportation have led to the growth of EVs in recent years.&nbsp;</p><p><br></p><p>In 2015, the Indian government launched the FAME India scheme to encourage the adoption of electric and hybrid vehicles.&nbsp;</p><p><br></p><ul><li>The EV market in India will experience a remarkable growth rate of 49% CAGR projected from 2021 to 2030. By 2030, it is estimated that the sales volume for this market segment will surpass 17 million units annually.</li><li>The sales of electric vehicles (EVs) in India have witnessed an 86% increase, from 2,37,811 units sold in 2021 to 4,42,901 units sold in 2022-23&nbsp;</li><li>In Budget 2023-24, ₹5,172 crores have been a

In [18]:
x = resp.json()

x["data"]["constituents"]

[{'sid': 'TTPW',
  'shares': 0.1745301968161239,
  'weight': 0.07121335466999999,
  'assetId': 'TTPW',
  'sidInfo': {'sector': 'Power Transmission & Distribution',
   'name': 'Tata Power Company Ltd',
   'ticker': 'TATAPOWER',
   'exchange': 'NSE',
   'description': 'The Tata Power Company Limited is an integrated power company engaged in generation, transmission, distribution and trading of \nelectricity. It is also engaged in mining and trading of coal.',
   'nseSeries': 'EQ',
   'nseTicker': 'TATAPOWER',
   'bseSeries': None,
   'bseTicker': 'TATAPOWER',
   'isin': 'INE245A01021',
   'type': 'stock'}},
 {'sid': 'EXID',
  'shares': 0.234342686513163,
  'weight': 0.07628107925000001,
  'assetId': 'EXID',
  'sidInfo': {'sector': 'Batteries',
   'name': 'Exide Industries Ltd',
   'ticker': 'EXIDEIND',
   'exchange': 'NSE',
   'description': 'Exide Industries Limited manufactures, markets and sells a range of lead acid storage batteries. The Company makes batteries for the automotive, po